# <font color=darkblue> Nettoyer Texte</font>

In [5]:
import pandas as pd
import html

#### <font color=black> Création de la fonction pour nettoyer</font>

In [6]:
#data_dir = '../../data/smarttweet/'
df = pd.read_csv('tweet2.csv')

In [15]:
# create dataframe
df.columns=['account', 'label', 'text', 'image_url']
df.image_url = df.image_url.apply(lambda url: url if type(url) is str else None)

# decode bytes for text column
def string_escape(s, encoding='utf-8'):
    return (s.encode('latin1')         # To bytes, required by 'unicode-escape'
             .decode('unicode-escape') # Perform the actual octal-escaping decode
             .encode('latin1')         # 1:1 mapping back to bytes
             .decode(encoding))        # Decode original encoding

def clean_text(text):
    # fix ascii encoding rendered as strings
    if( (text.startswith("b'") and text.endswith("'")) or (text.startswith('b"') and text.endswith('"'))):
        text = string_escape(text[2:-1])  # remove b'    '   or b"     "
    # remove URLs
    text = ' '.join([word for word in text.split() if not 'http' in word])
    # html unescape (&amp; and co)
    text = html.unescape(text)
    return text
    
df.text = df.text.apply(clean_text)
print(df.text)

0                         Better together: lace & cutouts.
1        Too much of a good thing? Never. Buy 3, get 3 ...
2        Does your bra collection need a refresh? Buy 2...
3                          Everything’s falling into lace.
4           New bra tops work double-duty. Just add jeans.
                               ...                        
16046    RT @Snoodit: Exclusive: #OUAT creators @AdamHo...
16047    The creator of #AmazonPrimeVideo's upcoming sc...
16048    Ben Affleck had a wonderful time playing Bruce...
16049    RT @TheRyanParker: Ben Affleck really did not ...
16050    RT @inkookang: i am not allowed to say what th...
Name: text, Length: 16051, dtype: object


In [17]:
from csv import writer
from csv import reader
 
def add_column_in_csv(input_file, output_file, transform_row):
    """ Append a column in existing csv using csv.reader / csv.writer classes"""
    # Open the input_file in read mode and output_file in write mode
    with open(input_file, 'r') as read_obj, \
            open(output_file, 'w', newline='') as write_obj:
        # Create a csv.reader object from the input file object
        csv_reader = reader(read_obj)
        # Create a csv.writer object from the output file object
        csv_writer = writer(write_obj)
        # Read each row of the input csv file as list
        for row in csv_reader:
            # Pass the list / row in the transform function to add column text for this row
            transform_row(row, csv_reader.line_num)
            # Write the updated row / list to the output file
            csv_writer.writerow(row)

#### <font color=black> Ajouter d'une colonne dans notre Csv avec texte nettoyé </font>

In [18]:
header_of_new_col = 'texte_N'
# Add the column in csv file with header
add_column_in_csv('tweet2.csv', 'tweet2_N.csv',
                  lambda row, line_num: row.append(header_of_new_col) if line_num == 1 else row.append(
                      clean_text(row[2])))
